# How to Scrape Data From Facebook Page Posts for Statistical Analysis

By [Max Woolf (@minimaxir)](http://minimaxir.com/)

This notebook describes how to build a Facebook Scraper using the latest version of Facebook's Graph API (v2.4). This is the accompanyment to my blog post [How to Scrape Data From Facebook Page Posts for Statistical Analysis](http://minimaxir.com/2015/07/facebook-scraper/).

In [10]:
# import some Python dependencies

import urllib2
import json
import datetime
import csv
import time

Accessing Facebook page data requires an access token.

Since the user access token expires within an hour, we need to create a dummy application *for the sole purpose of scraping* and use the app ID and app secret generated there [as described here](https://developers.facebook.com/docs/facebook-login/access-tokens#apptokens), both of which never expire.

In [11]:
# Since the code output in this notebook leaks the app_secret,
# it has been reset by the time you read this.

app_id = "1126944254025998"
app_secret = "abea2139d0f027f8db87850213c4e470" # DO NOT SHARE WITH ANYONE!

access_token = app_id + "|" + app_secret

Now we can access public Facebook data without limit. Let's do our analysis on the [New York Times Facebook page](https://www.facebook.com/nytimes), which is popular enough to yield good data.

In [12]:
page_id = 'DailyMail'

Let's write a quick program to ping NYT's Facebook page to verify that the `access_token` works and the `page_id` is valid.

In [13]:
def testFacebookPageData(page_id, access_token):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.4"
    node = "/" + page_id
    parameters = "/?access_token=%s" % access_token
    url = base + node + parameters
    
    # retrieve data
    req = urllib2.Request(url)
    response = urllib2.urlopen(req)
    data = json.loads(response.read())
    
    print json.dumps(data, indent=4, sort_keys=True)
    

testFacebookPageData(page_id, access_token)

{
    "id": "164305410295882", 
    "name": "Daily Mail"
}


When scraping large amounts of data from public APIs, there's a high probability that you'll hit an [HTTP Error 500 (Internal Error)](http://www.checkupdown.com/status/E500.html) at some point. There is no way to avoid that on our end. 

Instead, we'll use a helper function to catch the error and try again after a few seconds, which usually works. This helper function also consolidates the data retrival code, so it kills two birds with one stone.

In [14]:
def request_until_succeed(url):
    req = urllib2.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib2.urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception, e:
            print e
            time.sleep(5)
            
            print "Error for URL %s: %s" % (url, datetime.datetime.now())

    return response.read()

The data is the Facebook Page metadata however; we need to change the endpoint to the /feed endpoint.

In [15]:
def testFacebookPageFeedData(page_id, access_token):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.4"
    node = "/" + page_id + "/feed" # changed
    parameters = "/?access_token=%s" % access_token
    url = base + node + parameters
    
    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    print json.dumps(data, indent=4, sort_keys=True)
    

testFacebookPageFeedData(page_id, access_token)

{
    "data": [
        {
            "created_time": "2016-10-17T13:15:07+0000", 
            "id": "164305410295882_1554271377965938", 
            "message": "Not the hero anyone needed..."
        }, 
        {
            "created_time": "2016-10-17T13:00:51+0000", 
            "id": "164305410295882_1554230941303315", 
            "message": "'America and Russia WILL come to a point of war'"
        }, 
        {
            "created_time": "2016-10-17T12:45:07+0000", 
            "id": "164305410295882_1554200241306385", 
            "message": "Her son even gets mistaken for her BOYFRIEND!"
        }, 
        {
            "created_time": "2016-10-17T12:30:16+0000", 
            "id": "164305410295882_1554176171308792", 
            "message": "Amazing child. Well done!"
        }, 
        {
            "created_time": "2016-10-17T12:15:12+0000", 
            "id": "164305410295882_1554144001312009", 
            "message": "Pucker up!"
        }, 
        {
            "crea

In v2.4, the default behavior is to return very, very little metadata for statuses in order to reduce bandwidth, with the expectation the user will request the necessary fields.

We don't need data on every NYT status. Yet. Let's reduce the requested fields to exactly what we need, and the number of stories returned to 1 so we can process it.

In [16]:
def getFacebookPageFeedData(page_id, access_token, num_statuses):
    
    # construct the URL string
    base = "https://graph.facebook.com"
    node = "/" + page_id + "/feed" 
    parameters = "/?fields=message,link,created_time,type,name,id,likes.limit(1).summary(true),comments.limit(1).summary(true),shares&limit=%s&access_token=%s" % (num_statuses, access_token) # changed
    url = base + node + parameters
    
    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    return data
    

test_status = getFacebookPageFeedData(page_id, access_token, 1)["data"][0]
print json.dumps(test_status, indent=4, sort_keys=True)

{
    "comments": {
        "data": [
            {
                "created_time": "2016-10-17T13:20:34+0000", 
                "from": {
                    "id": "1796613387276055", 
                    "name": "Craig Butler"
                }, 
                "id": "1554271377965938_1554282307964845", 
                "message": "At least he didn't try to save an imaginary dog from a fire"
            }
        ], 
        "paging": {
            "cursors": {
                "after": "MjkZD", 
                "before": "MjkZD"
            }, 
            "next": "https://graph.facebook.com/v2.8/164305410295882_1554271377965938/comments?access_token=1126944254025998%7Cabea2139d0f027f8db87850213c4e470&summary=true&limit=1&after=MjkZD"
        }, 
        "summary": {
            "can_comment": false, 
            "order": "ranked", 
            "total_count": 29
        }
    }, 
    "created_time": "2016-10-17T13:15:07+0000", 
    "id": "164305410295882_1554271377965938", 
    "lik

Now that we have a sample Facebook page status, we can write a function to process each field individually.

In [17]:
def processFacebookPageFeedStatus(status):
    
    # The status is now a Python dictionary, so for top-level items,
    # we can simply call the key.
    
    # Additionally, some items may not always exist,
    # so must check for existence first
    
    status_id = status['id']
    status_message = '' if 'message' not in status.keys() else status['message'].encode('utf-8')
    link_name = '' if 'name' not in status.keys() else status['name'].encode('utf-8')
    status_type = status['type']
    status_link = '' if 'link' not in status.keys() else status['link']
    
    
    # Time needs special care since a) it's in UTC and
    # b) it's not easy to use in statistical programs.
    
    status_published = datetime.datetime.strptime(status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published + datetime.timedelta(hours=-5) # EST
    status_published = status_published.strftime('%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs
    
    # Nested items require chaining dictionary keys.
    
    num_likes = 0 if 'likes' not in status.keys() else status['likes']['summary']['total_count']
    num_comments = 0 if 'comments' not in status.keys() else status['comments']['summary']['total_count']
    num_shares = 0 if 'shares' not in status.keys() else status['shares']['count']
    
    # return a tuple of all processed data
    return (status_id, status_message, link_name, status_type, status_link,
           status_published, num_likes, num_comments, num_shares)

processed_test_status = processFacebookPageFeedStatus(test_status)
print processed_test_status

(u'164305410295882_1554271377965938', 'Not the hero anyone needed...', "Man 'high on LSD' breaks into house to save dog from imaginary fire", u'link', u'http://dailym.ai/2dVFBJW', '2016-10-17 08:15:07', 208, 29, 51)


Surprisingly, we're almost done! Now we just need to:

1. Query each page of Facebook Page Statuses (100 statuses per page) using `getFacebookPageFeedData`.
2. Process all statuses on that page using `processFacebookPageFeedStatus` and writing the output to a CSV file.
3. Navigate to the next page, and repeat until no more statuses

This block implements both the writing to CSV and page navigation.

In [20]:
def scrapeFacebookPageFeedStatus(page_id, access_token):
    with open('%s_facebook_statuses.csv' % page_id, 'wb') as file:
        w = csv.writer(file)
        w.writerow(["status_id", "status_message", "link_name", "status_type", "status_link",
           "status_published", "num_likes", "num_comments", "num_shares"])
        
        has_next_page = True
        num_processed = 0   # keep a count on how many we've processed
        scrape_starttime = datetime.datetime.now()
        
        print "Scraping %s Facebook Page: %s\n" % (page_id, scrape_starttime)
        
        statuses = getFacebookPageFeedData(page_id, access_token, 100)
        
        while has_next_page:
            for status in statuses['data']:
                w.writerow(processFacebookPageFeedStatus(status))
                
                # output progress occasionally to make sure code is not stalling
                num_processed += 1
                if num_processed % 1000 == 0:
                    print "%s Statuses Processed: %s" % (num_processed, datetime.datetime.now())
                    
            # if there is no next page, we're done.
            if 'paging' in statuses.keys():
                statuses = json.loads(request_until_succeed(statuses['paging']['next']))
            else:
                has_next_page = False
                
        
        print "\nDone!\n%s Statuses Processed in %s" % (num_processed, datetime.datetime.now() - scrape_starttime)


scrapeFacebookPageFeedStatus(page_id, access_token)

Scraping DailyMail Facebook Page: 2016-10-17 16:51:56.155208

1000 Statuses Processed: 2016-10-17 16:52:36.162065
2000 Statuses Processed: 2016-10-17 16:53:15.609109
3000 Statuses Processed: 2016-10-17 16:53:57.217003
4000 Statuses Processed: 2016-10-17 16:54:38.241483
5000 Statuses Processed: 2016-10-17 16:55:22.099445
6000 Statuses Processed: 2016-10-17 16:56:07.078425
7000 Statuses Processed: 2016-10-17 16:56:41.495671


KeyError: 'total_count'

The CSV can be opened in all major statistical programs. Have fun! :)

You can download the [NYTimes data here](https://dl.dropboxusercontent.com/u/2017402/nytimes_facebook_statuses.zip). [4.6MB]